In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 10 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

26 December 2022 ; 14:19:28


In [3]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,12,0,5.8,1.1,2.0,0.542,...,2.52,113.63,111.03,2.60,2.73,113.80,110.96,2.84,1,1
11,MarJon Beauchamp,SF,22,MIL,22,8,15.2,2.3,5.5,0.417,...,2.52,113.63,111.03,2.60,2.73,113.80,110.96,2.84,1,1
17,A.J. Lawson,SG,22,MIN,1,0,2.0,1.0,1.0,1.000,...,-0.91,113.05,113.93,-0.88,-1.37,112.83,114.16,-1.33,1,1
25,Josh Minott,SF,20,MIN,6,0,2.5,0.5,0.8,0.600,...,-0.91,113.05,113.93,-0.88,-1.37,112.83,114.16,-1.33,1,1
34,Wendell Moore Jr.,SG,21,MIN,16,2,6.0,0.8,1.8,0.464,...,-0.91,113.05,113.93,-0.88,-1.37,112.83,114.16,-1.33,1,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,34.5,ORL,17.6,1.518519,4.859259,7.2,21.7,12,0.587515
1,Bennedict Mathurin,28.5,IND,13.1,-0.745455,2.236364,5.5,17.2,7,0.127606
2,Walker Kessler,18.2,UTA,20.8,1.537500,6.150000,2.9,6.8,7,0.118185
3,Keegan Murray,28.8,SAC,10.0,-0.282759,1.979310,4.3,11.9,6,0.103289
4,Jaden Ivey,30.3,DET,12.6,-0.529032,0.000000,5.5,15.5,15,0.101329


In [7]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)